In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indian-liver-patient-records/indian_liver_patient.csv


In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

In [3]:
df = pd.read_csv('../input/indian-liver-patient-records/indian_liver_patient.csv')

In [6]:
print(df.head())

   Age  Gender  Total_Bilirubin  Direct_Bilirubin  Alkaline_Phosphotase  \
0   65  Female              0.7               0.1                   187   
1   62    Male             10.9               5.5                   699   
2   62    Male              7.3               4.1                   490   
3   58    Male              1.0               0.4                   182   
4   72    Male              3.9               2.0                   195   

   Alamine_Aminotransferase  Aspartate_Aminotransferase  Total_Protiens  \
0                        16                          18             6.8   
1                        64                         100             7.5   
2                        60                          68             7.0   
3                        14                          20             6.8   
4                        27                          59             7.3   

   Albumin  Albumin_and_Globulin_Ratio  Dataset  
0      3.3                        0.90        1 

In [7]:
# Check for missing values
print(df.isnull().sum())

Age                           0
Gender                        0
Total_Bilirubin               0
Direct_Bilirubin              0
Alkaline_Phosphotase          0
Alamine_Aminotransferase      0
Aspartate_Aminotransferase    0
Total_Protiens                0
Albumin                       0
Albumin_and_Globulin_Ratio    4
Dataset                       0
dtype: int64


In [21]:
df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})

In [22]:
df = df.fillna(df.mean())

In [24]:
X = df.drop('Dataset', axis=1)
y = df['Dataset']

In [25]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [30]:
# Initialize the Random Forest model(TRAIN MODEL)
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [31]:
X_train_df = pd.DataFrame(X_train)

In [34]:
X_train_clean = X_train_df.dropna()
y_train_clean = y_train[X_train_df.index.isin(X_train_clean.index)]

In [35]:
print("X_train_clean shape:", X_train_clean.shape)
print("y_train_clean shape:", y_train_clean.shape)

X_train_clean shape: (0, 10)
y_train_clean shape: (0,)


In [36]:
print("X_train_clean is empty:", X_train_clean.empty)
print("y_train_clean is empty:", y_train_clean.empty)

X_train_clean is empty: True
y_train_clean is empty: True


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
print("NaN values in X_train_clean:", np.isnan(X_train_clean).any())
print("Infinite values in X_train_clean:", np.isinf(X_train_clean).any())

NaN values in X_train_clean: Age                           False
Gender                        False
Total_Bilirubin               False
Direct_Bilirubin              False
Alkaline_Phosphotase          False
Alamine_Aminotransferase      False
Aspartate_Aminotransferase    False
Total_Protiens                False
Albumin                       False
Albumin_and_Globulin_Ratio    False
dtype: bool
Infinite values in X_train_clean: Age                           False
Gender                        False
Total_Bilirubin               False
Direct_Bilirubin              False
Alkaline_Phosphotase          False
Alamine_Aminotransferase      False
Aspartate_Aminotransferase    False
Total_Protiens                False
Albumin                       False
Albumin_and_Globulin_Ratio    False
dtype: bool


In [44]:
X_train_df = pd.DataFrame(X_train)
y_train_df = pd.Series(y_train)

In [49]:
# Check for missing values
print("Missing values in X_train:")
print(X_train_df.isna().sum())

Missing values in X_train:
Age                             0
Gender                        466
Total_Bilirubin                 0
Direct_Bilirubin                0
Alkaline_Phosphotase            0
Alamine_Aminotransferase        0
Aspartate_Aminotransferase      0
Total_Protiens                  0
Albumin                         0
Albumin_and_Globulin_Ratio      0
dtype: int64


In [50]:
print("Missing values in y_train:")
print(y_train_df.isna().sum())

Missing values in y_train:
0


In [53]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

Shape of X_train: (466, 10)
Shape of y_train: (466,)


In [54]:
X_train_df = pd.DataFrame(X_train)

In [55]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer

# Assume X_train is a DataFrame
X_train_df = pd.DataFrame(X_train)

# Separate features with numeric types and categorical types if needed
numeric_features = X_train_df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train_df.select_dtypes(include=['object']).columns

# Create imputers for different types of data
numeric_transformer = SimpleImputer(strategy='mean')
categorical_transformer = SimpleImputer(strategy='most_frequent')

# Combine transformers in a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Create a pipeline with preprocessing and model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Train the model
pipeline.fit(X_train_df, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  Index(['Age', 'Gender', 'Total_Bilirubin', 'Direct_Bilirubin',
       'Alkaline_Phosphotase', 'Alamine_Aminotransferase',
       'Aspartate_Aminotransferase', 'Total_Protiens', 'Albumin',
       'Albumin_and_Globulin_Ratio'],
      dtype='object')),
                                                 ('cat',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  Index([], dtype='object'))])),
                ('classifier', RandomForestClassifier())])

In [58]:
# Check for missing values in target
print("Missing values in y_train:", y_train_df.isna().sum())

Missing values in y_train: 0


In [64]:
X_train_clean = X_train_df[~y_train_df.isna()]
y_train_clean = y_train_df[~y_train_df.isna()]

In [66]:
print(X_train_clean)

     Age  Gender  Total_Bilirubin  Direct_Bilirubin  Alkaline_Phosphotase  \
77    68     NaN              0.6               0.1                  1620   
581   31     NaN              1.3               0.5                   184   
210   28     NaN              0.8               0.3                   190   
192   60     NaN              2.3               0.6                   272   
449   48     NaN              0.9               0.2                   173   
..   ...     ...              ...               ...                   ...   
71    75     NaN              0.8               0.2                   188   
106   36     NaN              5.3               2.3                   145   
270   37     NaN              0.7               0.2                   235   
435   17     NaN              0.5               0.1                   206   
102   17     NaN              0.9               0.2                   224   

     Alamine_Aminotransferase  Aspartate_Aminotransferase  Total_Protiens  

In [67]:
print(y_train_clean)

77     1
581    1
210    1
192    1
449    1
      ..
71     1
106    2
270    1
435    2
102    1
Name: Dataset, Length: 466, dtype: int64


In [71]:
target_imputer = SimpleImputer(strategy='most_frequent')
y_train_clean = target_imputer.fit_transform(y_train_df.values.reshape(-1, 1)).ravel()

In [72]:
model = HistGradientBoostingClassifier()

In [74]:
# Train the model
model.fit(X_train_clean, y_train_clean)

HistGradientBoostingClassifier()

In [76]:
y_pred = model.predict(X_test)

In [77]:
# Evaluate the model
print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy Score: 0.7606837606837606
Classification Report:
               precision    recall  f1-score   support

           1       0.84      0.84      0.84        87
           2       0.53      0.53      0.53        30

    accuracy                           0.76       117
   macro avg       0.69      0.69      0.69       117
weighted avg       0.76      0.76      0.76       117

